<img align="left" width="40%" src="http://www.lsce.ipsl.fr/Css/img/banniere_LSCE_75.png">

<br>Patrick BROCKMANN - LSCE (Climate and Environment Sciences Laboratory)
<hr>

### Discover Milankovitch Orbital Parameters over Time by reproducing figure from https://biocycle.atmos.colostate.edu/shiny/Milankovitch/

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot, column
from bokeh.models import CustomJS, Slider, RangeSlider
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [2]:
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact

In [3]:
import pandas as pd
import numpy as np

### Download files

Data files from http://vo.imcce.fr/insola/earth/online/earth/earth.html

In [4]:
! wget -nc http://vo.imcce.fr/insola/earth/online/earth/La2004/INSOLN.LA2004.BTL.100.ASC
! wget -nc http://vo.imcce.fr/insola/earth/online/earth/La2004/INSOLP.LA2004.BTL.ASC

Fichier «INSOLN.LA2004.BTL.100.ASC» déjà présent ; pas de récupération.

Fichier «INSOLP.LA2004.BTL.ASC» déjà présent ; pas de récupération.



### Read files

In [5]:
# t        Time from J2000  in 1000 years
# e        eccentricity
# eps      obliquity (radians)
# pibar    longitude of perihelion from moving equinox (radians)

df1 = pd.read_csv('INSOLN.LA2004.BTL.250.ASC', delim_whitespace=True, names=['t', 'e', 'eps', 'pibar'])
df1.set_index('t', inplace=True)

df2 = pd.read_csv('INSOLP.LA2004.BTL.ASC', delim_whitespace=True, names=['t', 'e', 'eps', 'pibar'])
df2.set_index('t', inplace=True)

#df = pd.read_csv('La2010a_ecc3.dat', delim_whitespace=True, names=['t', 'e'])
#df = pd.read_csv('La2010a_alkhqp3L.dat', delim_whitespace=True, names=['t','a','l','k','h','q','p'])

In [6]:
# INSOLP.LA2004.BTL.ASC has a FORTRAN DOUBLE notation D instead of E

df2['e'] = df2['e'].str.replace('D','E')
df2['e'] = df2['e'].astype(float)

df2['eps'] = df2['eps'].str.replace('D','E')
df2['eps'] = df2['eps'].astype(float)

df2['pibar'] = df2['pibar'].str.replace('D','E')
df2['pibar'] = df2['pibar'].astype(float)

df2['e'][0]

0.01670236225492288

In [7]:
df = pd.concat([df1[::-1],df2[1:]])
df

,e,eps,pibar
t,,,
-249000.0,0.017722,0.388669,5.878995
-248999.0,0.017862,0.387563,0.010612
-248998.0,0.018027,0.386791,0.426305
-248997.0,0.018357,0.386396,0.840109
-248996.0,0.018625,0.386407,1.250226
...,...,...,...
20996.0,0.025473,0.394611,4.074812
20997.0,0.026019,0.393511,4.359345
20998.0,0.026633,0.392498,4.647594


In [8]:
# t        Time from J2000  in 1000 years
# e        eccentricity
# eps      obliquity (radians)
# pibar    longitude of perihelion from moving equinox (radians)

df['eccentricity'] = df['e']
df['perihelion'] = df['pibar']
df['obliquity'] = 180. * df['eps'] / np.pi

df['precession'] = df['eccentricity'] * np.sin(df['perihelion'])


#latitude <- 65. * pi / 180.
#Q.day <- S0*(1+eccentricity*sin(perihelion+pi))^2 *sin(latitude)*sin(obliquity) 

latitude = 65. * np.pi / 180.
df['insolation'] =  1367 * ( 1 + df['eccentricity'] * np.sin(df['perihelion'] + np.pi))**2 * np.sin(latitude) * np.sin(df['eps'])   
    
df

,e,eps,pibar,eccentricity,perihelion,obliquity,precession,insolation
t,,,,,,,,
-249000.0,0.017722,0.388669,5.878995,0.017722,5.878995,22.269093,-0.006970,476.065726
-248999.0,0.017862,0.387563,0.010612,0.017862,0.010612,22.205724,0.000190,468.052656
-248998.0,0.018027,0.386791,0.426305,0.018027,0.426305,22.161492,0.007454,460.402978
-248997.0,0.018357,0.386396,0.840109,0.018357,0.840109,22.138860,0.013671,454.212985
-248996.0,0.018625,0.386407,1.250226,0.018625,1.250226,22.139490,0.017676,450.543600
...,...,...,...,...,...,...,...,...
20996.0,0.025473,0.394611,4.074812,0.025473,4.074812,22.609562,-0.020469,496.001285
20997.0,0.026019,0.393511,4.359345,0.026019,4.359345,22.546515,-0.024414,498.523231
20998.0,0.026633,0.392498,4.647594,0.026633,4.647594,22.488504,-0.026577,499.408912


### Build plot

In [9]:
a = widgets.IntRangeSlider(
    layout=Layout(width='600px'),
    value=[-2000, 50],
    min=-250000,
    max=21000,
    step=100,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    description='-249Myr to +21Myr:',
)

In [10]:
def plot1(limits):
    
    years = df[limits[0]:limits[1]].index
    
    zeroSpan = Span(location=0, dimension='height', line_color='black',
                    line_dash='solid', line_width=1)
    
    p1 = figure(title='Eccentricity', active_scroll="wheel_zoom")
    p1.line(years, df[limits[0]:limits[1]]['eccentricity'], color='red')  
    p1.yaxis.axis_label = "Degrees"
    p1.add_layout(zeroSpan)

    p2 = figure(title='Obliquity', x_range=p1.x_range)
    p2.line(years, df[limits[0]:limits[1]]['obliquity'], color='forestgreen')
    p2.yaxis.axis_label = "Degrees"
    p2.add_layout(zeroSpan)
 
    p3 = figure(title='Precessional index', x_range=p1.x_range)
    p3.line(years, df[limits[0]:limits[1]]['precession'], color='dodgerblue')
    p3.yaxis.axis_label = "Degrees"
    p3.add_layout(zeroSpan)
    
    p4 = figure(title='Mean Daily Insolation at 65N on Summer Solstice', x_range=p1.x_range)
    p4.line(years, df[limits[0]:limits[1]]['insolation'], color='#ffc125')
    p4.yaxis.axis_label = "Watts/m2"
    p4.add_layout(zeroSpan)
    
    show(gridplot([p1,p2,p3,p4], ncols=1, plot_width=600, plot_height=200))
    
interact(plot1, limits=a)

interactive(children=(IntRangeSlider(value=(-2000, 50), continuous_update=False, description='-249Myr to +21My…

<function __main__.plot1(limits)>

In [11]:
# Merged tool of subfigures is not marked as active
# https://github.com/bokeh/bokeh/issues/10659

p1 = figure(title='Eccentricity', active_scroll="wheel_zoom")
years = df[0:2000].index
p1.line(years, df[0:2000]['eccentricity'], color='red')

p2 = figure(title='Obliquity', x_range=p1.x_range)
p2.line(years, df[0:2000]['obliquity'], color='forestgreen')

show(gridplot([p1,p2], ncols=1, plot_width=600, plot_height=200, merge_tools=True))